<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-02 14:26:42
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.33 C
-------------------
Today PnL: 44.26 K (0.34%)
Current PnL: -15.88 L (-11.07%)
CY Booked + Current PnL: -11.41 L (-7.95%)
-------------------
Total profit:  4.32 L
Total loss:  -20.21 L
-------------------
Total Booked + Current PnL: 15.57 L (13.36%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.00 L (59.61%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.40 C (105.86%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.2%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-2.96,H-LC,92.47,238489.0,43797.0,5294.0,0.75,22.50,2.22,25.21,29.0,8.27,1.86,31.88,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,82.80,104228.0,25866.0,8161.0,2.46,33.01,7.83,43.43,71.0,3.17,0.81,39.40,XY24,NTT,MISC
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,173688.0,32848.0,11637.0,4.07,23.32,6.70,31.59,76.0,2.82,1.35,50.87,XY24,NTT,BANKS
62,RELIANCE,1291.83,1526.0,3.85,X-LC,17.20,154082.0,13273.0,12250.0,-0.51,9.43,7.95,18.13,17.0,1.08,1.20,21.66,XY25,NTT,REFINERIES
68,SBIN,760.30,863.0,-11.57,M-LC,40.86,215869.0,14390.0,12823.0,0.28,7.14,5.94,13.51,73.0,1.12,1.68,20.64,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.78,H-SC,53.76,116614.0,14610.0,30413.0,5.14,14.32,26.08,44.14,111.0,0.48,0.91,32.89,XR,ATH,MISC
75,SURYODAY,189.78,240.00,64.47,H-SC,76.34,116549.0,-33377.0,73053.0,5.05,-22.26,62.68,26.46,129.0,-0.46,0.91,49.31,XR,NTT,BANKS
85,UNIONBANK,123.87,163.00,-0.31,M-LC,91.40,173688.0,32848.0,11637.0,4.07,23.32,6.70,31.59,76.0,2.82,1.35,50.87,XY24,NTT,BANKS
63,REPCOHOME,515.07,880.00,-56.78,H-SC,37.63,160593.0,-44405.0,189644.0,3.34,-21.66,118.09,70.85,125.0,-0.23,1.25,29.64,XY24,NTT,FINANCE
84,UJJIVANSFB,52.77,60.00,38.86,M-SC,45.16,119988.0,-22491.0,42008.0,3.28,-15.79,35.01,13.70,217.0,-0.54,0.94,41.62,X40N,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,7.53,61459.0,-52090.0,92139.0,-3.44,-45.87,149.92,35.27,238.0,-0.57,0.48,50.42,XR,NTT,HOTELS
82,TRIDENT,37.35,48.00,58.66,M-SC,87.10,76249.0,-16006.0,42311.0,-2.89,-17.35,55.49,28.51,220.0,-0.38,0.59,30.47,XR,NTT,TEXTILES
23,DEN,52.79,75.00,109.84,L-SC,79.57,58976.0,-25488.0,61022.0,-2.64,-30.18,103.47,42.07,235.0,-0.42,0.46,25.25,AR,NTT,ENTERTAINMENT
70,SHALBY,261.39,327.00,836.20,M-SC,12.90,127256.0,-53887.0,99361.0,-2.06,-29.75,78.08,25.10,204.0,-0.54,0.99,1.85,XY24,NTT,HEALTHCARE
33,HEROMOTOCO,4311.81,6039.03,-3.33,H-MC,74.19,147858.0,-3055.0,63505.0,-1.97,-2.02,42.95,40.06,89.0,-0.05,1.15,20.94,AR,ATH,AUTO


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.50,X-LC,43.01,246752.0,16153.0,114962.0,-0.69,7.00,46.59,56.86,1.0,0.14,1.92,14.21,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.44,X-LC,22.58,237657.0,-27376.0,107207.0,-0.55,-10.33,45.11,30.12,2.0,-0.26,1.85,6.59,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.53,X-LC,70.97,183636.0,-16674.0,75474.0,0.63,-8.32,41.10,29.35,4.0,-0.22,1.43,26.87,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-9.25,X-LC,73.12,82125.0,-28047.0,117118.0,0.59,-25.46,142.61,80.85,5.0,-0.24,0.64,20.96,AR,ATH,ELECTRICAL
32,HAVELLS,1588.02,2069.16,-8.59,X-LC,10.75,236293.0,-11438.0,86507.0,-0.81,-4.62,36.61,30.30,6.0,-0.13,1.84,8.06,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,11.94,H-SC,97.85,129375.0,31728.0,15952.0,0.21,32.49,12.33,48.82,138.0,1.99,1.01,43.72,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,173688.0,32848.0,11637.0,4.07,23.32,6.70,31.59,76.0,2.82,1.35,50.87,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,82.80,104228.0,25866.0,8161.0,2.46,33.01,7.83,43.43,71.0,3.17,0.81,39.40,XY24,NTT,MISC
49,LICI,785.05,983.0,-2.96,H-LC,92.47,238489.0,43797.0,5294.0,0.75,22.50,2.22,25.21,29.0,8.27,1.86,31.88,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3576.0,-16.79,X-LC,32.26,92217.0,17095.0,15059.0,-1.07,22.76,16.33,42.81,11.0,1.14,0.72,15.65,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,38.95,M-SC,67.74,143830.0,6252.0,88153.0,2.35,4.54,61.29,68.62,182.0,0.07,1.12,24.69,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,11.94,H-SC,97.85,129375.0,31728.0,15952.0,0.21,32.49,12.33,48.82,138.0,1.99,1.01,43.72,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,9.15,H-SC,89.25,161084.0,23224.0,14546.0,0.56,16.85,9.03,27.39,118.0,1.60,1.26,34.78,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,6.01,H-MC,54.84,100734.0,15633.0,27470.0,-1.09,18.37,27.27,50.65,92.0,0.57,0.79,26.63,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,3.96,M-LC,82.80,104228.0,25866.0,8161.0,2.46,33.01,7.83,43.43,71.0,3.17,0.81,39.40,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.78,H-SC,53.76,116614.0,14610.0,30413.0,5.14,14.32,26.08,44.14,111.0,0.48,0.91,32.89,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,55.91,142239.0,13087.0,73979.0,2.50,10.13,52.01,67.42,155.0,0.18,1.11,38.34,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-10.16,M-LC,33.33,153642.0,2697.0,106750.0,-0.75,1.79,69.48,72.51,69.0,0.03,1.20,15.36,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,35.48,123458.0,122.0,136841.0,-0.13,0.10,110.84,111.05,108.0,0.00,0.96,20.97,AR,ATH,MISC
1,AARTIIND,487.04,919.00,37.02,M-SC,66.67,78657.0,-2679.0,74819.0,0.37,-3.29,95.12,88.69,202.0,-0.04,0.61,32.64,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-57.75,L-SC,7.53,61459.0,-52090.0,92139.0,-3.44,-45.87,149.92,35.27,238.0,-0.57,0.48,50.42,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-22.76,H-SC,68.82,87675.0,-10305.0,32177.0,-0.76,-10.52,36.70,22.32,130.0,-0.32,0.68,27.16,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,82.80,104228.0,25866.0,8161.0,2.46,33.01,7.83,43.43,71.0,3.17,0.81,39.40,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-378.23,H-SC,100.00,118209.0,-15077.0,99674.0,-0.25,-11.31,84.32,63.47,131.0,-0.15,0.92,98.15,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,45.16,119988.0,-22491.0,42008.0,3.28,-15.79,35.01,13.70,217.0,-0.54,0.94,41.62,X40N,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-378.23,H-SC,100.00,118209.0,-15077.0,99674.0,-0.25,-11.31,84.32,63.47,131.0,-0.15,0.92,98.15,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,6616.67,M-SC,98.92,73695.0,-20115.0,96732.0,-1.48,-21.44,131.26,81.67,203.0,-0.21,0.57,42.71,XR,NTT,CERAMICS
55,PFIZER,4245.50,6318.30,11.94,H-SC,97.85,129375.0,31728.0,15952.0,0.21,32.49,12.33,48.82,138.0,1.99,1.01,43.72,X40,ATH,PHARMA
77,TANLA,917.30,1963.11,-32.99,H-SC,96.77,163895.0,-75520.0,348474.0,1.36,-31.54,212.62,114.01,110.0,-0.22,1.28,50.32,AR,ATH,IT
74,STARHEALTH,551.56,761.00,4.66,M-SC,95.70,171118.0,-29098.0,105118.0,-1.31,-14.53,61.43,37.97,194.0,-0.28,1.33,37.39,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.86
1,25,41.21
2,50,70.07


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.12
XY25    14.42
X40     14.28
X40N    13.54
AR      12.45
XR      10.60
X200     1.43
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.65
M-SC    17.77
X-LC    17.30
H-MC     8.70
M-LC     7.86
H-LC     7.31
X-MC     5.64
M-MC     5.60
X-SC     2.80
L-SC     2.36
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.47,-0.36,31.93
BANKS,9.96,-9.64,60.04
IT,9.90,-17.92,82.37
MISC,6.49,-29.95,87.43
FINANCE,5.95,-24.98,84.32
INSURANCE,5.44,4.47,24.16
PAINTS,5.38,-19.09,54.33
ELECTRICAL,4.97,-5.22,56.81
AUTO,4.41,-10.45,58.69


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2853360.0,26
AR,1250990.0,12
X40N,1210846.0,17
XR,1090530.0,14
X40,769010.0,11
XY25,452790.0,10
SR,197284.0,2
X200,75474.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2241246.0,21
M-SC,2033957.0,21
X-LC,799572.0,12
M-MC,549083.0,5
H-MC,511938.0,8
X-SC,350016.0,4
X-MC,309640.0,5
M-LC,296086.0,6
L-SC,295120.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      931384.0      6
M-SC       XY24      752120.0      6
H-SC       AR        544432.0      3
           X40N      499615.0      6
M-SC       X40N      467339.0      6
X-LC       X40       458575.0      5
M-MC       XY24      394426.0      3
M-SC       XR        292270.0      4
H-LC       AR        263643.0      3
H-SC       XR        235317.0      4
M-SC       X40       221698.0      2
M-LC       XY24      176513.0      4
H-MC       XY24      175503.0      2
L-SC       XR        153980.0      2
X-LC       AR        143579.0      2
X-MC       XY24      139897.0      2
X-SC       X40N      122010.0      2
X-LC       XY25      121944.0      4
M-SC       XY25      121241.0      1
X-SC       XR        115880.0      1
H-MC       XR        112354.0      1
X-SC       SR        112126.0      1
M-LC       XR        106750.0      1
X-MC       X40N       94412.0      2
M-SC       AR         94131.0      1
           SR         85158.0      1
M-MC       AR         80678.0      1
L-SC       XY24       80118.0      1
X-LC       X200       75474.0      1
X-MC       XY25       75331.0      1
H-MC       XY25       74867.0      1
M-MC       XR         73979.0      1
H-MC       AR         63505.0      1
L-SC       AR         61022.0      1
H-MC       X40        58239.0      2
L-LC       XY25       41290.0      1
H-SC       X40        30498.0      2
H-MC       X40N       27470.0      1
M-LC       XY25       12823.0      1
H-LC       XY25        5294.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
